In [1]:
#import de todas las librerias necesarias de python y spark
import pyspark
import re
import pyspark.sql.functions as f
from pyspark.sql import SparkSession

from pyspark.sql.functions import col,udf,asc,lower
from pyspark.sql.types import IntegerType

from pyspark.ml.feature import Tokenizer,RegexTokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import HashingTF,IDF,Tokenizer

#Creación de la sesión para trabajar en spark
spark= SparkSession.builder.appName('proyecto3').getOrCreate()

In [2]:
#Creación de los tres dataframes a utilizar
df1= spark.read.csv('/FileStore/tables/articles1.csv',inferSchema=True,header=True)
df2= spark.read.csv('/FileStore/tables/articles2.csv',inferSchema=True,header=True)
df3= spark.read.csv('/FileStore/tables/articles3.csv',inferSchema=True,header=True)

#unión de los tres dataframes en uno solo
dfx=df1.union(df2)
dft=dfx.union(df3)

In [3]:
dft.show()

+---+-----+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
_c0| id| title| publication| author| date| year|month| url| content|
+---+-----+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
 0|17283|House Republicans...|New York Times| Carl Hulse|2016-12-31|2016.0| 12.0|null|WASHINGTON — C...|
 1|17284|Rift Between Offi...|New York Times|Benjamin Mueller ...|2017-06-19|2017.0| 6.0|null|After the bullet ...|
 2|17285|Tyrus Wong, ‘Bamb...|New York Times| Margalit Fox|2017-01-06|2017.0| 1.0|null|When Walt Disney’...|
 3|17286|Among Deaths in 2...|New York Times| William McDonald|2017-04-10|2017.0| 4.0|null|Death may be the ...|
 4|17287|Kim Jong-un Says ...|New York Times| Choe Sang-Hun|2017-01-02|2017.0| 1.0|null|SEOUL, South Kore...|
 5|17288|Sick With a Cold,...|New York Times| Sewell Chan|2017-01-02|2017.0| 1.0|null|LONDON — Queen...|
 6|17289|Taiwan’s Presiden...|New York Times| Javier C. Hernández|2017-01-02|2017.0| 1.0|null|BEIJING — Pres...|
 7|17290|After ‘The Bigges...|New York Times| Gina Kolata|2017-02-08|2017.0| 2.0|null|Danny Cahill stoo...|
 8|17291|First, a Mixtape....|New York Times| Katherine Rosman|2016-12-31|2016.0| 12.0|null|Just how is Hil...|
 9|17292|Calling on Angels...|New York Times| Andy Newman|2016-12-31|2016.0| 12.0|null|Angels are everyw...|
 10|17293|Weak Federal Powe...|New York Times| Justin Gillis|2017-01-03|2017.0| 1.0|null|With Donald J. Tr...|
 11|17294|Can Carbon Captur...|New York Times| John Schwartz|2017-01-05|2017.0| 1.0|null|THOMPSONS, Tex. ...|
 12|17295|Mar-a-Lago, the F...|New York Times| Maggie Haberman|2017-01-02|2017.0| 1.0|null|WEST PALM BEACH, ...|
 13|17296|How to form healt...|New York Times| Charles Duhigg|2017-01-02|2017.0| 1.0|null|This article is p...|
 14|17297|Turning Your Vaca...|New York Times|Stephanie Rosenbloom|2017-04-14|2017.0| 4.0|null|It’s the season f...|
 15|17298|As Second Avenue ...|New York Times| Emma G. Fitzsimmons|2017-01-02|2017.0| 1.0|null|Finally. The Seco...|
 16|17300|Dylann Roof Himse...|New York Times|Kevin Sack and Al...|2017-01-02|2017.0| 1.0|null| pages into the ...|
 17|17301|Modi’s Cash Ban B...|New York Times| Geeta Anand|2017-01-02|2017.0| 1.0|null|MUMBAI, India — ...|
 18|17302|Suicide Bombing i...|New York Times|The Associated Press|2017-01-03|2017.0| 1.0|null|BAGHDAD — A su...|
 19|17303|Fecal Pollution T...|New York Times| Brett Cole|2017-01-03|2017.0| 1.0|null|SYDNEY, Australia...|
+---+-----+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
only showing top 20 rows

In [4]:
#separación de las tablas en el contenido interesante para la solución del problema
df=dft.select(dft['id'],dft['title'],dft['content'])

In [5]:
df.show()

+-----+--------------------+--------------------+
 id| title| content|
+-----+--------------------+--------------------+
17283|House Republicans...|WASHINGTON — C...|
17284|Rift Between Offi...|After the bullet ...|
17285|Tyrus Wong, ‘Bamb...|When Walt Disney’...|
17286|Among Deaths in 2...|Death may be the ...|
17287|Kim Jong-un Says ...|SEOUL, South Kore...|
17288|Sick With a Cold,...|LONDON — Queen...|
17289|Taiwan’s Presiden...|BEIJING — Pres...|
17290|After ‘The Bigges...|Danny Cahill stoo...|
17291|First, a Mixtape....|Just how is Hil...|
17292|Calling on Angels...|Angels are everyw...|
17293|Weak Federal Powe...|With Donald J. Tr...|
17294|Can Carbon Captur...|THOMPSONS, Tex. ...|
17295|Mar-a-Lago, the F...|WEST PALM BEACH, ...|
17296|How to form healt...|This article is p...|
17297|Turning Your Vaca...|It’s the season f...|
17298|As Second Avenue ...|Finally. The Seco...|
17300|Dylann Roof Himse...| pages into the ...|
17301|Modi’s Cash Ban B...|MUMBAI, India — ...|
17302|Suicide Bombing i...|BAGHDAD — A su...|
17303|Fecal Pollution T...|SYDNEY, Australia...|
+-----+--------------------+--------------------+
only showing top 20 rows

In [6]:
#creación de los tokenizadores de palabras, en este caso se crean dos tipos, pero el tipo a utilizar es el que permite 
#las expresiones regulares regex_tokenizer, estas permiten limpiar caracteres especiales y palabras de longitud 1 
tokenizer= Tokenizer(inputCol='content', outputCol='words')
regex_tokenizer= RegexTokenizer(inputCol='content', outputCol='words',pattern='[^a-zA-Z0-9]+').setMinTokenLength(2)
count_tokens= udf(lambda words:len(words),IntegerType())

In [7]:
#limpieza de los datos nulos en contenido y titulo, además de la transformación del dataframe en tokens
df2=df.fillna("unknown", subset=["content","title"])
tokenized= regex_tokenizer.transform(df2)

In [8]:
tokenized.show()

+-----+--------------------+--------------------+--------------------+
 id| title| content| words|
+-----+--------------------+--------------------+--------------------+
17283|House Republicans...|WASHINGTON — C...|[washington, cong...|
17284|Rift Between Offi...|After the bullet ...|[after, the, bull...|
17285|Tyrus Wong, ‘Bamb...|When Walt Disney’...|[when, walt, disn...|
17286|Among Deaths in 2...|Death may be the ...|[death, may, be, ...|
17287|Kim Jong-un Says ...|SEOUL, South Kore...|[seoul, south, ko...|
17288|Sick With a Cold,...|LONDON — Queen...|[london, queen, e...|
17289|Taiwan’s Presiden...|BEIJING — Pres...|[beijing, preside...|
17290|After ‘The Bigges...|Danny Cahill stoo...|[danny, cahill, s...|
17291|First, a Mixtape....|Just how is Hil...|[just, how, is, h...|
17292|Calling on Angels...|Angels are everyw...|[angels, are, eve...|
17293|Weak Federal Powe...|With Donald J. Tr...|[with, donald, tr...|
17294|Can Carbon Captur...|THOMPSONS, Tex. ...|[thompsons, tex, ...|
17295|Mar-a-Lago, the F...|WEST PALM BEACH, ...|[west, palm, beac...|
17296|How to form healt...|This article is p...|[this, article, i...|
17297|Turning Your Vaca...|It’s the season f...|[it, the, season,...|
17298|As Second Avenue ...|Finally. The Seco...|[finally, the, se...|
17300|Dylann Roof Himse...| pages into the ...|[pages, into, the...|
17301|Modi’s Cash Ban B...|MUMBAI, India — ...|[mumbai, india, i...|
17302|Suicide Bombing i...|BAGHDAD — A su...|[baghdad, suicide...|
17303|Fecal Pollution T...|SYDNEY, Australia...|[sydney, australi...|
+-----+--------------------+--------------------+--------------------+
only showing top 20 rows

In [9]:
#removedor de stopwords en ingles, creando un nuevo dataframe sin las mismas
remover= StopWordsRemover(inputCol='words', outputCol='filtered')
sinsw=remover.transform(tokenized)

In [10]:
sinsw.show()

+-----+--------------------+--------------------+--------------------+--------------------+
 id| title| content| words| filtered|
+-----+--------------------+--------------------+--------------------+--------------------+
17283|House Republicans...|WASHINGTON — C...|[washington, cong...|[washington, cong...|
17284|Rift Between Offi...|After the bullet ...|[after, the, bull...|[bullet, shells, ...|
17285|Tyrus Wong, ‘Bamb...|When Walt Disney’...|[when, walt, disn...|[walt, disney, ba...|
17286|Among Deaths in 2...|Death may be the ...|[death, may, be, ...|[death, may, grea...|
17287|Kim Jong-un Says ...|SEOUL, South Kore...|[seoul, south, ko...|[seoul, south, ko...|
17288|Sick With a Cold,...|LONDON — Queen...|[london, queen, e...|[london, queen, e...|
17289|Taiwan’s Presiden...|BEIJING — Pres...|[beijing, preside...|[beijing, preside...|
17290|After ‘The Bigges...|Danny Cahill stoo...|[danny, cahill, s...|[danny, cahill, s...|
17291|First, a Mixtape....|Just how is Hil...|[just, how, is, h...|[hillary, kerr, f...|
17292|Calling on Angels...|Angels are everyw...|[angels, are, eve...|[angels, everywhe...|
17293|Weak Federal Powe...|With Donald J. Tr...|[with, donald, tr...|[donald, trump, t...|
17294|Can Carbon Captur...|THOMPSONS, Tex. ...|[thompsons, tex, ...|[thompsons, tex, ...|
17295|Mar-a-Lago, the F...|WEST PALM BEACH, ...|[west, palm, beac...|[west, palm, beac...|
17296|How to form healt...|This article is p...|[this, article, i...|[article, part, s...|
17297|Turning Your Vaca...|It’s the season f...|[it, the, season,...|[season, family, ...|
17298|As Second Avenue ...|Finally. The Seco...|[finally, the, se...|[finally, second,...|
17300|Dylann Roof Himse...| pages into the ...|[pages, into, the...|[pages, journal, ...|
17301|Modi’s Cash Ban B...|MUMBAI, India — ...|[mumbai, india, i...|[mumbai, india, b...|
17302|Suicide Bombing i...|BAGHDAD — A su...|[baghdad, suicide...|[baghdad, suicide...|
17303|Fecal Pollution T...|SYDNEY, Australia...|[sydney, australi...|[sydney, australi...|
+-----+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [11]:
#Testeo de hash y frecuencia de documento inverso
#esto no se utilizo al final pero es interesante verlo
#hashing_tf= HashingTF(inputCol='filtered',outputCol='rawFeatures')
#featurized_data=hashing_tf.transform(sinsw)
#featurized_data.show()

In [12]:
#idf=IDF(inputCol='rawFeatures', outputCol='resultado')
#idf_model=idf.fit(featurized_data)
#idf_model=idf.fit(featurized_data)
#rescaled_data=idf_model.transform(featurized_data)
#rescaled_data.select('id','title','filtered','resultado').show()

In [13]:
#creación y selección de la estructura del indice inverso, por medio de una agrupación y una agregación de las palabras tokenizadas
wordcount = sinsw.select("id", f.explode("filtered").alias("words")).groupBy("id","words").count().groupBy("words").agg(f.collect_list(f.struct(f.col("id"), f.col("count"))).alias("wordcount"))
IndiceInverso = wordcount.orderBy('words').sort(asc('words'))

In [14]:
IndiceInverso.show()

+--------------+--------------------+
 words| wordcount|
+--------------+--------------------+
 00|[[138704, 1], [14...|
 000|[[49331, 1], [495...|
 0000|[[175877, 1], [64...|
 00000|[[171552, 1], [17...|
 000000001| [[174523, 1]]|
 00000001| [[169599, 1]]|
 000000031| [[172701, 1]]|
 0000001mm| [[158088, 1]]|
 0000019047619| [[199816, 1]]|
00000259067358| [[199816, 1]]|
 000003| [[195911, 1]]|
 00001|[[37097, 1], [700...|
 00002| [[198348, 1]]|
 00003| [[65725, 1]]|
 0000364334| [[24915, 1]]|
 000038| [[202117, 1]]|
 00004|[[65244, 1], [191...|
 00009| [[69156, 1]]|
 0001|[[67303, 1], [226...|
 0002|[[77081, 1], [267...|
+--------------+--------------------+
only showing top 20 rows

In [15]:
#creación del widget que ofrece databricks para introducir el texto
dbutils.widgets.text("word","Inserte palabra a buscar"," ")

In [16]:
#Muestra de los 5 textos con la mayor cantidad de las palabras
dfI= IndiceInverso.filter(IndiceInverso['words'] == dbutils.widgets.get("word").lower()).collect()
contador =dfI[0].wordcount
contador = sorted(contador, key=lambda x: x[1])
arregloTitulos = list(reversed(contador))[:5]

In [17]:
for x in range (0,5):
  print(str(arregloTitulos[x][1])+", "+sinsw.filter(sinsw['id'] == arregloTitulos[x][0]).collect()[0].id+", "+sinsw.filter(sinsw['id'] == arregloTitulos[x][0]).collect()[0].title)

126, 78594, A Sit-In on the House Floor Over Gun Control 
46, 202036, Nancy Pelosi says a Clinton landslide could put the House in play. The math backs her up.
45, 201907, Democrats really might have a shot at taking the House. Here’s the math.
45, 28005, Exclusive — Audio Emerges of When Paul Ryan Abandoned Donald Trump: ‘I Am Not Going to Defend Donald Trump—Not Now, Not in the Future’
45, 27430, Exclusive — Discussion About GOP Replacement to Paul Ryan as Speaker of the House Intensifies in White House, Congress - Breitbart